## BLERSSI Pipeline Deployment in UCS

### Clone git repo

In [1]:
! git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 4142 (delta 51), reused 123 (delta 35), pack-reused 3980
Receiving objects: 100% (4142/4142), 15.17 MiB | 43.15 MiB/s, done.
Resolving deltas: 100% (1555/1555), done.


### Import libraries

In [13]:
import kfp
import os
from kubernetes import client
import calendar
import time

### Component files Declarations

In [73]:
path='cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem/pipelines/'
component_root_katib= path+'components/v2/tf-katib/'
component_root_train= path+'components/v2/tf-model-train/'
component_root_serve = path+'components/v2/tf-serving-deploy-service/'
component_root_webui = path+'components/v2/tf-webui-deploy-service/'
component_root_tensorboard= path+'components/v2/tf-tensorboard-deploy-service/'

#### Components Description

tf_katib_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp;&nbsp;&nbsp; Run katib for BLERSSI to get optimal hyperparamater values and use it to train model. <br>
tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train BLERSSI model  and save model in nfs-volume. <br> 
tf_serve_model_op &nbsp; - &nbsp; Load BLERSSI model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
tf_webui_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp; Web application client UI loads Unlabeled data, does preprocessing, send API request to serving and display Predicted Location. <br> 
tf_tensor_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  - &nbsp; Tensorboard UI which provides the visualization metrics such as loss and accuracy, model graph (ops and layers) and histograms of weights, biases, or other tensors for BLERSSI Model <br> 

### Load cpu or gpu component yaml

In [74]:
processor = "CPU" # Provide GPU or CPU as variable to include which processor to use
if processor == "GPU" : component_file="component_gpu.yaml"
else : component_file="component_cpu.yaml"
    
processor

'CPU'

### Load components from respective .YAML config files

In [75]:
tf_katib_op = kfp.components.load_component_from_file(os.path.join(component_root_katib, 'component.yaml')) 
tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component_cpu.yaml')) 
tf_serve_model_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))
tf_webui_op = kfp.components.load_component_from_file(os.path.join(component_root_webui, 'component.yaml'))
tf_tensor_op = kfp.components.load_component_from_file(os.path.join(component_root_tensorboard, 'component.yaml'))

### Define Volume and Volume Mounts

In [76]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [77]:
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
def blerssi_pipeline():

    #Defining Task to run katib
    tf_katib_task = tf_katib_op(timestamp=timestamp)
    tf_katib_task.add_volume(nfs_volume)
    tf_katib_task.add_volume_mount(nfs_volume_mount)    
    
        
    #Defining Task for Model training
    tf_train_model_task = tf_train_model_op(timestamp=timestamp)
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    tf_train_model_task.after(tf_katib_task)
    
    #Defining Task to perform serving    
    tf_serve_model_task = tf_serve_model_op(timestamp=timestamp)
    tf_serve_model_task.add_volume(nfs_volume)
    tf_serve_model_task.add_volume_mount(nfs_volume_mount)
    tf_serve_model_task.after(tf_train_model_task)

    #Defining task for tensorboard creation    
    tf_tensorboard_task = tf_tensor_op(timestamp=timestamp)
    tf_tensorboard_task.add_volume(nfs_volume)
    tf_tensorboard_task.add_volume_mount(nfs_volume_mount)
    tf_tensorboard_task.after(tf_train_model_task)
    
    
    #Defining task for webUI creation
    tf_webui_task = tf_webui_op(image_path="docker.io/premkarthi/blerssi-webui:v4",
                                timestamp=timestamp)
    tf_webui_task.add_volume(nfs_volume)
    tf_webui_task.add_volume_mount(nfs_volume_mount)
    tf_webui_task.after(tf_serve_model_task)
    
#Creating a pipeline run
kfp.Client().create_run_from_pipeline_func(blerssi_pipeline, arguments={})

1592937370


RunPipelineResult(run_id=c87f3894-e36b-41f1-99ed-3ac913f7441a)

### Before accessing service URLs below, make sure that Pipeline Run is Complete in the Dashboard

### Get Tensorboard URL

In [9]:
TB_SERVICE_NAME = "tensorboard-"+str(timestamp)+"-blerssi-service"
port = !(kubectl get svc {TB_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('Tensorboard service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

Tensorboard service is available at - http://INGRESS_IP:30436


### Get WebUI URL 
And upload csv from https://github.com/CiscoAI/kubeflow-apps/blob/master/tf-blerssi/data/iBeacon_RSSI_Unlabeled.csv

In [10]:
WEBUI_SERVICE_NAME = "webui-"+str(timestamp)+"-blerssi-service"
port = !(kubectl get svc {WEBUI_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('WebUI service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

WebUI service is available at - http://INGRESS_IP:31805
